In [2]:
import pandas as pd
import numpy as np
import csv
from collections import OrderedDict
import os

In [3]:
path = '/ctao_shared_data/Liver_optum/outputs/'
ot_path = '../Data_V3/'

In [4]:
"""
Method: create_map
Map two lists into a zipped list
"""
def create_map (a, b):
    zipMap = list(zip(list(a), list(b)))
    return zipMap

In [5]:
## Import files

log_file = []
log_df_len = []
log_clm_unq = []
log_enc_unq = []
log_pt_unq = []
clm_ls = []
#clmid_ls = []
#enc_ls = []

# path = '/data/user_data/xinyue_AD/aha/' ## input path
# ot_path = '/data/user_data/xinyue_AD/Preprocess/Jingna/' ## output path



special = ['final_drug_all', 'final_proc_all_original']

#file_ls = ['final_diag_all','final_drug_all','final_lab','final_proc_all']

file_ls = ['final_diag_all','final_drug_all','final_proc_all']
#file_ls = ['final_diag_all','final_drug_all','final_lab','final_proc_all_original']

cl = ['patid','fst_dt']

for f in file_ls:
    if f in special:
        df = pd.read_csv(path+f+'.csv', delimiter = ',', low_memory = False, usecols=cl)
    else:
        df = pd.read_csv(path+f+'.csv', delimiter = '\t', low_memory = False, usecols=cl)
    log_pt_unq.append(df.patid.nunique())
    
    df['encounter_id']=df.patid.astype(str)+'_'+df.fst_dt.astype(str)
    df = df.drop_duplicates()
    log_file.append(f)
    log_df_len.append(len(df))
    log_clm_unq.append(df.patid.nunique())
    log_enc_unq.append(df.encounter_id.nunique())
    
    new_ls = create_map(df.patid, df.fst_dt)
    clm_ls += new_ls
    print(clm_ls[0], clm_ls[len(clm_ls)-1], len(clm_ls))
    del new_ls

(560499200082764, '2017-09-07') (560499899999890, '2015-02-11') 3609211
(560499200082764, '2017-09-07') (560499899999890, '2010-05-24') 6413456
(560499200082764, '2017-09-07') (560499899999890, '2015-02-11') 9779264


In [6]:
log_df = pd.DataFrame({'file':log_file,'df size':log_df_len,'patid count':log_clm_unq,'encounter count':log_enc_unq,'patient count':log_pt_unq})

In [7]:
log_df

,file,df size,patid count,encounter count,patient count
0,final_diag_all,3609211,25531,3609211,25531
1,final_drug_all,2804245,23992,2804245,23992
2,final_proc_all,3365808,24693,3365808,24693


In [8]:
total = log_df.sum()
total.name = '5'
log_df = log_df.append(total.transpose())
log_df

/tmp/ipykernel_813347/1950329238.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  log_df = log_df.append(total.transpose())


,file,df size,patid count,encounter count,patient count
0,final_diag_all,3609211,25531,3609211,25531
1,final_drug_all,2804245,23992,2804245,23992
2,final_proc_all,3365808,24693,3365808,24693
5,final_diag_allfinal_drug_allfinal_proc_all,9779264,74216,9779264,74216


In [9]:
clm_df = pd.DataFrame(clm_ls,columns = ['patid','fst_dt'])
clm_df = clm_df.drop_duplicates()
clm_df['encounter_id']=clm_df.patid.astype(str)+'_'+clm_df.fst_dt.astype(str)
clm_df = clm_df[['encounter_id','patid','fst_dt']]
clm_df = clm_df.sort_values(by=['encounter_id']).reset_index(drop=True)
clm_df

,encounter_id,patid,fst_dt
0,560499200082764_2017-06-13,560499200082764,2017-06-13
1,560499200082764_2017-07-14,560499200082764,2017-07-14
2,560499200082764_2017-08-22,560499200082764,2017-08-22
3,560499200082764_2017-09-07,560499200082764,2017-09-07
4,560499200082764_2017-09-26,560499200082764,2017-09-26
...,...,...,...
5771662,560499899999890_2015-02-06,560499899999890,2015-02-06
5771663,560499899999890_2015-02-07,560499899999890,2015-02-07
5771664,560499899999890_2015-02-09,560499899999890,2015-02-09
5771665,560499899999890_2015-02-10,560499899999890,2015-02-10


In [10]:
clm_df['patid'].value_counts()

560499829592146    3368
560499824160667    3173
560499812789874    3073
560499806818276    2869
560499847668700    2642
                   ... 
560499292719878       1
560499405665590       1
560499492994737       1
560499405633942       1
560499896123814       1
Name: patid, Length: 25531, dtype: int64

In [22]:
# clm_df[clm_df['patid']==560499200217248]
# clm_df[clm_df['patid']==560499829592146]

In [12]:
clm_df.to_csv(ot_path+'dict_encounter.csv',index=False)

# Create dictionaries and codes for using by main code to generate .tsv

## Diagnosis

In [5]:
f = "final_diag_all"
diagnosis = pd.read_csv(path+f+'.csv', delimiter = '\t', low_memory = False)

In [6]:
diagnosis.head()

,patid,fst_dt,icd_flag,diag
0,560499200082764,2017-09-07,10,Z944
1,560499200082764,2017-12-21,10,Z4823
2,560499200082764,2017-12-21,10,Z944
3,560499200082764,2017-12-21,10,E7223
4,560499200082764,2017-12-21,10,Z48298


In [7]:
diagnosis[diagnosis.diag == '3720']

,patid,fst_dt,icd_flag,diag
16002783,560499899955561,2009-09-06,9,3720
16002789,560499899955561,2009-09-07,9,3720
16002793,560499899955561,2009-09-08,9,3720
16002801,560499899955561,2009-09-09,9,3720
16002803,560499899955561,2009-09-10,9,3720


In [8]:
diagnosis['encounter_id'] = diagnosis.patid.astype(str)+'_'+diagnosis.fst_dt.astype(str)

In [9]:
diagnosis['modified_diag'] = np.where((diagnosis['icd_flag']==9), 'D_9_' + diagnosis['diag'].replace('.', ''), 'D_10_' + diagnosis['diag'].replace('.', ''))

In [10]:
diagnosis

,patid,fst_dt,icd_flag,diag,encounter_id,modified_diag
0,560499200082764,2017-09-07,10,Z944,560499200082764_2017-09-07,D_10_Z944
1,560499200082764,2017-12-21,10,Z4823,560499200082764_2017-12-21,D_10_Z4823
2,560499200082764,2017-12-21,10,Z944,560499200082764_2017-12-21,D_10_Z944
3,560499200082764,2017-12-21,10,E7223,560499200082764_2017-12-21,D_10_E7223
4,560499200082764,2017-12-21,10,Z48298,560499200082764_2017-12-21,D_10_Z48298
...,...,...,...,...,...,...
16016911,560499899999890,2015-02-11,9,78079,560499899999890_2015-02-11,D_9_78079
16016912,560499899999890,2015-02-11,9,78605,560499899999890_2015-02-11,D_9_78605
16016913,560499899999890,2015-02-11,9,3569,560499899999890_2015-02-11,D_9_3569
16016914,560499899999890,2015-02-11,9,V5869,560499899999890_2015-02-11,D_9_V5869


In [11]:
diagnosis.icd_flag.value_counts()

10    8260060
9     7756856
Name: icd_flag, dtype: int64

In [12]:
diagnosis.isnull().values.any()

False

In [13]:
cl = ['patid', 'encounter_id', 'modified_diag']
diagnosis = diagnosis[cl]

In [19]:
diagnosis[diagnosis.modified_diag == 'D_9_3720']

,patid,encounter_id,modified_diag
16002783,560499899955561,560499899955561_2009-09-06,D_9_3720
16002789,560499899955561,560499899955561_2009-09-07,D_9_3720
16002793,560499899955561,560499899955561_2009-09-08,D_9_3720
16002801,560499899955561,560499899955561_2009-09-09,D_9_3720
16002803,560499899955561,560499899955561_2009-09-10,D_9_3720


In [26]:
diagnosis.to_csv(ot_path+'mod_diagnosis.csv',index=False)

In [33]:
#diag_id_code_dic = diagnosis['modified_diag'].drop_duplicates().reset_index(drop=True).to_dict()

In [34]:
#diag_id_code_dic

## Labs

In [35]:
#f = "final_lab"
#lab = pd.read_csv(path+f+'.csv', delimiter = '\t', low_memory = False)

In [36]:
#lab.head()

# Medications

In [31]:
# Load drugs file
f ='final_drug_all'
drug = pd.read_csv(path+f+'.csv', low_memory = False)
drug

,patid,fst_dt,ndc,ndc_qty
0,560499200082764,2017-06-13,16729004201,120.0
1,560499200082764,2017-06-13,16729004101,60.0
2,560499200082764,2017-06-13,00093733401,120.0
3,560499200082764,2017-07-14,16729004201,120.0
4,560499200082764,2017-07-14,00093733401,120.0
...,...,...,...,...
4694843,560499899999890,2010-05-19,67253026510,102.0
4694844,560499899999890,2010-05-19,00093001298,68.0
4694845,560499899999890,2010-05-19,00781122810,68.0
4694846,560499899999890,2010-05-24,00591317104,4.0


In [32]:
drug[drug.ndc == 'UNK']

,patid,fst_dt,ndc,ndc_qty
1000,560499200734235,2019-04-24,UNK,400.0
2414,560499201881476,2019-08-28,UNK,2.0
7970,560499202287750,2015-09-14,UNK,120.0
11579,560499202536587,2014-02-11,UNK,30.0
11586,560499202536587,2014-04-14,UNK,30.0
...,...,...,...,...
4682141,560499899830036,2010-05-27,UNK,2.0
4682172,560499899830036,2010-08-20,UNK,2.0
4687760,560499899881880,2011-09-19,UNK,0.5
4689089,560499899887357,2010-01-27,UNK,100.0


In [39]:
#drug = drug[drug.ndc != 'UNK'] #Added by Ahmed 11/22

In [40]:
# # Load drugs dictionary
# f ='dict_drug'
# dict_drug_df = pd.read_csv(path+f +'.csv', low_memory = False)
# dict_drug_df

In [41]:
# dict_drug_df.isna().any()

In [42]:
# nan_values = dict_drug_df[dict_drug_df['gnrc_nm'].isna()]
# nan_values

In [43]:
# dict_drug_df = dict_drug_df.dropna()
# dict_drug_df.info()

In [33]:
ndc = pd.read_csv(ot_path + 'lu_ndc.csv',delimiter = '\t',header=1,low_memory=False)
#ndc.info()
ndc = ndc.loc[ndc.AHFSCLSS!='UNK']
ndc = ndc[['NDC','GNRC_NM']].drop_duplicates().reset_index(drop=True)
ndc

,NDC,GNRC_NM
0,00000000888,NaN
1,00001900201,NaN
2,00001900301,NaN
3,00001900401,NaN
4,00001900501,NaN
...,...,...
437601,99496052102,HERBAL DRUGS
437602,99999999901,NaN
437603,99999999902,NaN
437604,99999999999,NaN


In [34]:
nan_values = ndc[ndc['GNRC_NM'].isna()]
ndc = ndc.dropna()
ndc[ndc.NDC == "16729004201"]

,NDC,GNRC_NM
171646,16729004201,TACROLIMUS


In [35]:
# One method to merge two data frames
ndc_gnrc_nm = dict(zip(ndc['NDC'], ndc['GNRC_NM'])) 
# {'16729004101': 'TACROLIMUS',
#  '16729004201': 'TACROLIMUS',
gnrc_nm_list = []
for ndc in drug['ndc']:
    if ndc in ndc_gnrc_nm:
        gnrc_nm_list.append(ndc_gnrc_nm[ndc])
    else:
        gnrc_nm_list.append(np.nan)
drug['gnrc_nm']= gnrc_nm_list
drug

,patid,fst_dt,ndc,ndc_qty,gnrc_nm
0,560499200082764,2017-06-13,16729004201,120.0,TACROLIMUS
1,560499200082764,2017-06-13,16729004101,60.0,TACROLIMUS
2,560499200082764,2017-06-13,00093733401,120.0,MYCOPHENOLATE MOFETIL
3,560499200082764,2017-07-14,16729004201,120.0,TACROLIMUS
4,560499200082764,2017-07-14,00093733401,120.0,MYCOPHENOLATE MOFETIL
...,...,...,...,...,...
4694843,560499899999890,2010-05-19,67253026510,102.0,CLONIDINE HCL
4694844,560499899999890,2010-05-19,00093001298,68.0,PANTOPRAZOLE SODIUM
4694845,560499899999890,2010-05-19,00781122810,68.0,METOPROLOL TARTRATE
4694846,560499899999890,2010-05-24,00591317104,4.0,ALENDRONATE SODIUM


In [36]:
drug[drug.ndc == 'UNK']

,patid,fst_dt,ndc,ndc_qty,gnrc_nm
1000,560499200734235,2019-04-24,UNK,400.0,NaN
2414,560499201881476,2019-08-28,UNK,2.0,NaN
7970,560499202287750,2015-09-14,UNK,120.0,NaN
11579,560499202536587,2014-02-11,UNK,30.0,NaN
11586,560499202536587,2014-04-14,UNK,30.0,NaN
...,...,...,...,...,...
4682141,560499899830036,2010-05-27,UNK,2.0,NaN
4682172,560499899830036,2010-08-20,UNK,2.0,NaN
4687760,560499899881880,2011-09-19,UNK,0.5,NaN
4689089,560499899887357,2010-01-27,UNK,100.0,NaN


In [37]:
nan_values = drug[drug['gnrc_nm'].isna()]
nan_values

,patid,fst_dt,ndc,ndc_qty,gnrc_nm
1000,560499200734235,2019-04-24,UNK,400.00,NaN
1960,560499201227386,2019-04-15,99999999692,1.00,NaN
2414,560499201881476,2019-08-28,UNK,2.00,NaN
4266,560499202063941,2019-04-04,000199092D0,0.00,NaN
4267,560499202063941,2019-04-04,00019909220,0.00,NaN
...,...,...,...,...,...
4688536,560499899885690,2013-07-02,00090019016,-10.00,NaN
4689089,560499899887357,2010-01-27,UNK,100.00,NaN
4689178,560499899887357,2011-02-21,UNK,100.00,NaN
4690643,560499899951947,2012-03-17,0,-10.00,NaN


In [38]:
drug = drug.dropna()
drug

,patid,fst_dt,ndc,ndc_qty,gnrc_nm
0,560499200082764,2017-06-13,16729004201,120.0,TACROLIMUS
1,560499200082764,2017-06-13,16729004101,60.0,TACROLIMUS
2,560499200082764,2017-06-13,00093733401,120.0,MYCOPHENOLATE MOFETIL
3,560499200082764,2017-07-14,16729004201,120.0,TACROLIMUS
4,560499200082764,2017-07-14,00093733401,120.0,MYCOPHENOLATE MOFETIL
...,...,...,...,...,...
4694843,560499899999890,2010-05-19,67253026510,102.0,CLONIDINE HCL
4694844,560499899999890,2010-05-19,00093001298,68.0,PANTOPRAZOLE SODIUM
4694845,560499899999890,2010-05-19,00781122810,68.0,METOPROLOL TARTRATE
4694846,560499899999890,2010-05-24,00591317104,4.0,ALENDRONATE SODIUM


In [39]:
drug.isna().any()

patid      False
fst_dt     False
ndc        False
ndc_qty    False
gnrc_nm    False
dtype: bool

In [40]:
drug[drug.ndc == 'UNK']

,patid,fst_dt,ndc,ndc_qty,gnrc_nm


In [41]:
drug['encounter_id'] = drug.patid.astype(str)+'_'+drug.fst_dt.astype(str)
mod_gen_drug_list = []
for gnrc_nm in drug.gnrc_nm:
    modified_name = 'M_' + gnrc_nm.lower().replace('.', '')
    mod_gen_drug_list.append(modified_name)
drug['mod_gen_drug'] = mod_gen_drug_list
drug

/tmp/ipykernel_813347/2559045116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug['encounter_id'] = drug.patid.astype(str)+'_'+drug.fst_dt.astype(str)
/tmp/ipykernel_813347/2559045116.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drug['mod_gen_drug'] = mod_gen_drug_list


,patid,fst_dt,ndc,ndc_qty,gnrc_nm,encounter_id,mod_gen_drug
0,560499200082764,2017-06-13,16729004201,120.0,TACROLIMUS,560499200082764_2017-06-13,M_tacrolimus
1,560499200082764,2017-06-13,16729004101,60.0,TACROLIMUS,560499200082764_2017-06-13,M_tacrolimus
2,560499200082764,2017-06-13,00093733401,120.0,MYCOPHENOLATE MOFETIL,560499200082764_2017-06-13,M_mycophenolate mofetil
3,560499200082764,2017-07-14,16729004201,120.0,TACROLIMUS,560499200082764_2017-07-14,M_tacrolimus
4,560499200082764,2017-07-14,00093733401,120.0,MYCOPHENOLATE MOFETIL,560499200082764_2017-07-14,M_mycophenolate mofetil
...,...,...,...,...,...,...,...
4694843,560499899999890,2010-05-19,67253026510,102.0,CLONIDINE HCL,560499899999890_2010-05-19,M_clonidine hcl
4694844,560499899999890,2010-05-19,00093001298,68.0,PANTOPRAZOLE SODIUM,560499899999890_2010-05-19,M_pantoprazole sodium
4694845,560499899999890,2010-05-19,00781122810,68.0,METOPROLOL TARTRATE,560499899999890_2010-05-19,M_metoprolol tartrate
4694846,560499899999890,2010-05-24,00591317104,4.0,ALENDRONATE SODIUM,560499899999890_2010-05-24,M_alendronate sodium


In [42]:
 drug[drug['mod_gen_drug'] == 'M_unknown']

,patid,fst_dt,ndc,ndc_qty,gnrc_nm,encounter_id,mod_gen_drug


In [43]:
cl = ['patid', 'encounter_id', 'mod_gen_drug']
drug = drug[cl]
drug.isna().any()

patid           False
encounter_id    False
mod_gen_drug    False
dtype: bool

In [44]:
drug.to_csv(ot_path+'mod_drug.csv',index=False)

# Procedures

In [45]:
f ='final_proc_all'
#f1 ='final_proc_all_original'
proc = pd.read_csv(path+f+'.csv', delimiter = '\t', low_memory = False)
proc.icd_flag.value_counts()

9     8244079
10    5334614
0         189
Name: icd_flag, dtype: int64

In [46]:
proc[proc.icd_flag == 0]

,patid,fst_dt,proc,icd_flag
37444,560499202520587,2014-08-28,36415,0
37453,560499202520587,2014-08-28,82248,0
37458,560499202520587,2014-08-28,84100,0
51751,560499202879182,2016-10-03,99285,0
158304,560499208877478,2017-12-27,K0738,0
...,...,...,...,...
9854686,560499868809724,2017-11-28,99214,0
10053159,560499874659178,2018-05-11,81003,0
12033592,560499892799912,2018-04-16,99214,0
12250000,560499893477054,2017-11-08,82977,0


In [58]:
#f ='final_proc_all'
# f ='final_proc_all_original'
# proc = pd.read_csv(path+f+'.csv', low_memory = False)
# proc.icd_flag.value_counts()

In [47]:
proc['encounter_id'] = proc.patid.astype(str)+'_'+proc.fst_dt.astype(str)
proc

,patid,fst_dt,proc,icd_flag,encounter_id
0,560499200082764,2017-09-07,83735,10,560499200082764_2017-09-07
1,560499200082764,2017-09-07,85610,10,560499200082764_2017-09-07
2,560499200082764,2017-09-07,82248,10,560499200082764_2017-09-07
3,560499200082764,2017-09-07,80053,10,560499200082764_2017-09-07
4,560499200082764,2017-09-07,36415,10,560499200082764_2017-09-07
...,...,...,...,...,...
13578877,560499899999890,2015-02-11,99238,9,560499899999890_2015-02-11
13578878,560499899999890,2015-02-11,A0425,9,560499899999890_2015-02-11
13578879,560499899999890,2015-02-11,E0439,9,560499899999890_2015-02-11
13578880,560499899999890,2015-02-11,A0428,9,560499899999890_2015-02-11


In [48]:
proc[proc['icd_flag'] ==0]

,patid,fst_dt,proc,icd_flag,encounter_id
37444,560499202520587,2014-08-28,36415,0,560499202520587_2014-08-28
37453,560499202520587,2014-08-28,82248,0,560499202520587_2014-08-28
37458,560499202520587,2014-08-28,84100,0,560499202520587_2014-08-28
51751,560499202879182,2016-10-03,99285,0,560499202879182_2016-10-03
158304,560499208877478,2017-12-27,K0738,0,560499208877478_2017-12-27
...,...,...,...,...,...
9854686,560499868809724,2017-11-28,99214,0,560499868809724_2017-11-28
10053159,560499874659178,2018-05-11,81003,0,560499874659178_2018-05-11
12033592,560499892799912,2018-04-16,99214,0,560499892799912_2018-04-16
12250000,560499893477054,2017-11-08,82977,0,560499893477054_2017-11-08


In [49]:
subset = proc[["icd_flag", "proc"]]
subset.drop_duplicates()


,icd_flag,proc
0,10,83735
1,10,85610
2,10,82248
3,10,80053
4,10,36415
...,...,...
13573152,10,0DQ10ZZ
13573157,10,0CUM07Z
13573200,10,42953
13575305,9,15933


In [50]:
subset['proc'].is_unique
dupdf= subset[subset.duplicated(['proc'])]
dupdf[dupdf.proc == '85025']

,icd_flag,proc
10,10,85025
183,10,85025
235,10,85025
265,10,85025
413,10,85025
...,...,...
13578768,9,85025
13578791,9,85025
13578808,9,85025
13578818,9,85025


In [63]:
#boolean = not proc[["patid", "proc"]].is_unique 

# boolean = proc.duplicated(subset=["icd_flag", "proc"]).any()
# boolean

# duplicateRowsDF = proc[proc.duplicated(['patid','proc', 'icd_flag'])]
# duplicateRowsDF

In [51]:
mod_proc_list = []
for proc_code in proc.proc:
    new_code = 'P_' + 'I_' + proc_code.replace('.', '')
    mod_proc_list.append(new_code)
proc['mod_proc_code'] = mod_proc_list
proc

,patid,fst_dt,proc,icd_flag,encounter_id,mod_proc_code
0,560499200082764,2017-09-07,83735,10,560499200082764_2017-09-07,P_I_83735
1,560499200082764,2017-09-07,85610,10,560499200082764_2017-09-07,P_I_85610
2,560499200082764,2017-09-07,82248,10,560499200082764_2017-09-07,P_I_82248
3,560499200082764,2017-09-07,80053,10,560499200082764_2017-09-07,P_I_80053
4,560499200082764,2017-09-07,36415,10,560499200082764_2017-09-07,P_I_36415
...,...,...,...,...,...,...
13578877,560499899999890,2015-02-11,99238,9,560499899999890_2015-02-11,P_I_99238
13578878,560499899999890,2015-02-11,A0425,9,560499899999890_2015-02-11,P_I_A0425
13578879,560499899999890,2015-02-11,E0439,9,560499899999890_2015-02-11,P_I_E0439
13578880,560499899999890,2015-02-11,A0428,9,560499899999890_2015-02-11,P_I_A0428


In [52]:
cl = ['patid', 'encounter_id', 'mod_proc_code']
proc = proc[cl]

In [53]:
proc.to_csv(ot_path+'mod_proc.csv',index=False)